In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv('/content/drive/MyDrive/oracle/ClosedStack_data_Virtual.csv', low_memory= False, encoding='latin-1')

In [4]:
df_new = df[[ 'Job ID','Hardware ID', 'Hardware Name', 'Solution ID', 'Ticket Site','Site Name', 'Job Created']]
df_filtered1 = df_new.dropna(subset=['Job ID', 'Job Created', 'Solution ID']).copy()
df_filtered1.loc[:, 'Job Created'] = pd.to_datetime(df_filtered1['Job Created']).dt.strftime('%Y-%m-%d')

In [29]:
data = {'Date': ['2019-01-01', '2019-01-07', '2019-01-14', '2019-01-21', '2019-01-28', '2019-02-04','2019-02-11', '2019-02-18', '2019-02-25','2019-03-04', '2019-03-11', '2019-03-18', '2019-03-25', '2019-04-01', '2019-04-07', '2019-04-14', '2019-04-21', '2019-04-28', '2019-05-05', '2019-05-12','2019-05-19','2019-05-26','2019-06-02','2019-06-09','2019-06-16','2019-06-23','2019-06-30',
                 '2019-07-07','2019-07-14','2019-07-21','2019-07-28','2019-08-04','2019-08-11','2019-08-18','2019-08-25','2019-09-01','2019-09-08','2019-09-15','2019-09-22','2019-09-29','2019-10-06','2019-10-13','2019-10-20','2019-10-27','2019-11-03',
                 '2019-11-10','2019-11-17','2019-11-24','2019-12-01','2019-12-08','2019-12-15','2019-12-22','2019-12-29'],
                      'Servers': [521, 1683, 2037, 974, 966, 1541, 1085, 1505, 1193, 1407, 878, 835, 593, 1184, 1265, 831, 686, 797, 1030, 1280, 1074, 618, 387, 446, 636, 548, 472, 863, 557, 904, 640, 655, 832, 606, 859,
                                  269, 839, 421, 400, 307, 535, 358, 467, 442, 324, 555, 490, 282, 357, 458, 359, 53, 142]}



df1 = pd.DataFrame(data)

print(df1)

          Date  Servers
0   2019-01-01      521
1   2019-01-07     1683
2   2019-01-14     2037
3   2019-01-21      974
4   2019-01-28      966
5   2019-02-04     1541
6   2019-02-11     1085
7   2019-02-18     1505
8   2019-02-25     1193
9   2019-03-04     1407
10  2019-03-11      878
11  2019-03-18      835
12  2019-03-25      593
13  2019-04-01     1184
14  2019-04-07     1265
15  2019-04-14      831
16  2019-04-21      686
17  2019-04-28      797
18  2019-05-05     1030
19  2019-05-12     1280
20  2019-05-19     1074
21  2019-05-26      618
22  2019-06-02      387
23  2019-06-09      446
24  2019-06-16      636
25  2019-06-23      548
26  2019-06-30      472
27  2019-07-07      863
28  2019-07-14      557
29  2019-07-21      904
30  2019-07-28      640
31  2019-08-04      655
32  2019-08-11      832
33  2019-08-18      606
34  2019-08-25      859
35  2019-09-01      269
36  2019-09-08      839
37  2019-09-15      421
38  2019-09-22      400
39  2019-09-29      307
40  2019-10-06  

In [21]:
df1.loc[:, 'Date'] =  pd.to_datetime(df1['Date']).dt.strftime('%Y-%m-%d')

In [ ]:
"Job ID","Job Status","Job Klass","Job Hardware ID","Job Operation","Job Created",
"Job Updated","Job Duration in Seconds","Job Ticket ID","Hardware ID","Hardware Name",
"Hardware Image ID","Hardware Solution ID","Hardware Site ID","Image ID","Image Name","Site ID",
"Site Name","Ticket Site Name","First Site Name","Solution ID","Solution Name","Ticket ID","Ticket Site"
,"Hardware Compute Config ID","Compute Config ID","Compute Config Name","Compute Config CPU","Compute Config Memory"
,"Compute Config Disk","Compute Config SKU"

#first site name, job created
#        1     2     3   4   5     6   7     8    9    10  11    12
#2019 : 5881 5409 3928 4392 4373 2018 3137 3250 2002 1939 1748 1299
#2020 : 2242 1688 1769 1817 1013 1496 1748 1521 1273 1377 1146 1159
#2021 : 1801 2211 1394 1169 1021 962 879 1163 1013 1111 752 359
#2022 : 839 1196 1223 1063 1159 1336 1094 999 1078 1269 1067 621


#2019:
# 1/1  1/7  1/14  1/21  1/28
# 521  1683  2037   974   966
# 2/4   2/11  2/18   2/25
# 1541  1085   1505  1193
# 3/4   3/11  3/18  3/25
# 1407  878   835   593
# 4/1  4/7   4/14  4/21  4/28
# 1184 1265  831   686   797
# 5/5  5/12  5/19  5/26
# 1030  1280  1074  618
# 6/2  6/9  6/16 6/23  6/30
# 387  446  636  548   472
# 7/7  7/14 7/21 7/28
# 863   557 904 640
# 8/4 8/11 8/18 8/25
# 655 832  606  859
# 9/1 9/8 9/15 9/22 9/29
# 269 839 421   400 307
# 10/6 10/13 10/20 10/27
# 535   358   467   442
# 11/3 11/10 11/17 11/24
# 324   555   490   282
# 12/1 12/8  12/15  12/22 12/29
# 357   458   359    53    142            TOTAL :39376


#2020:
# 1/5 1/12 1/19 1/26
# 758  521 455  438
# 2/2 2/9 2/16 2/23
# 326 403 516 443
# 3/1 3/8 3/15 3/22 3/29
# 478 305 563  340  413
# 4/5 4/12 4/19 4/26
# 675 485 180 165
# 5/3 5/10 5/17 5/24 5/31
# 143 237  295  320  439
# 6/7 6/14 6/21 6/28
# 448  435  140  184
# 7/5 7/12 7/19 7/26
# 326 407 533  332
# 8/2 8/9 8/16 8/23 8/30
# 244 424  439  350 226
# 9/6 9/13 9/20 9/27
# 378  308  239  303
# 10/4 10/11 10/18 10/25
# 321  416   166   357
# 11/1 11/8/ 11/15 11/22 11/29
# 241  412    343   90    242
# 12/6 12/13 12/20 12/27
# 373  352    181    73              TOTAL : 18249

In [ ]:
df_new2020 = df_new.loc[(df_new['Job Created'] <= "2021-01-01 00:00:00.000000") & (df_new['Job Created'] >= "2020-01-01 00:00:00.000000")]

df_new2020_1 = df_new.loc[(df_new['Job Created'] <= "2020-02-01 00:00:00.000000") & (df_new['Job Created'] >= "2020-01-01 00:00:00.000000")]
df_new2020_2 = df_new.loc[(df_new['Job Created'] <= "2020-03-01 00:00:00.000000") & (df_new['Job Created'] >= "2020-02-01 00:00:00.000000")]
df_new2020_3 = df_new.loc[(df_new['Job Created'] <= "2020-04-01 00:00:00.000000") & (df_new['Job Created'] >= "2020-03-01 00:00:00.000000")]
df_new2020_4 = df_new.loc[(df_new['Job Created'] <= "2020-05-01 00:00:00.000000") & (df_new['Job Created'] >= "2020-04-01 00:00:00.000000")]
df_new2020_5 = df_new.loc[(df_new['Job Created'] <= "2020-06-01 00:00:00.000000") & (df_new['Job Created'] >= "2020-05-01 00:00:00.000000")]
df_new2020_6 = df_new.loc[(df_new['Job Created'] <= "2020-07-01 00:00:00.000000") & (df_new['Job Created'] >= "2020-06-01 00:00:00.000000")]
df_new2020_7 = df_new.loc[(df_new['Job Created'] <= "2020-08-01 00:00:00.000000") & (df_new['Job Created'] >= "2020-07-01 00:00:00.000000")]
df_new2020_8 = df_new.loc[(df_new['Job Created'] <= "2020-09-01 00:00:00.000000") & (df_new['Job Created'] >= "2020-08-01 00:00:00.000000")]
df_new2020_9 = df_new.loc[(df_new['Job Created'] <= "2020-10-01 00:00:00.000000") & (df_new['Job Created'] >= "2020-09-01 00:00:00.000000")]
df_new2020_10 = df_new.loc[(df_new['Job Created'] <= "2020-11-01 00:00:00.000000") & (df_new['Job Created'] >= "2020-10-01 00:00:00.000000")]
df_new2020_11 = df_new.loc[(df_new['Job Created'] <= "2020-12-01 00:00:00.000000") & (df_new['Job Created'] >= "2020-11-01 00:00:00.000000")]
df_new2020_12 = df_new.loc[(df_new['Job Created'] <= "2021-01-01 00:00:00.000000") & (df_new['Job Created'] >= "2020-12-01 00:00:00.000000")]

df2020j = len(df_new2020_1)
df2020f = len(df_new2020_2)
df2020m = len(df_new2020_3)
df2020a = len(df_new2020_4)
df2020ma = len(df_new2020_5)
df2020ju = len(df_new2020_6)
df2020jy = len(df_new2020_7)
df2020au = len(df_new2020_8)
df2020s = len(df_new2020_9)
df2020o = len(df_new2020_10)
df2020n = len(df_new2020_11)
df2020d = len(df_new2020_12)

print(df2020j, df2020f, df2020m, df2020a, df2020ma, df2020ju, df2020jy, df2020au, df2020s, df2020o, df2020n,df2020d)

2242 1688 1769 1817 1013 1496 1748 1521 1273 1377 1146 1159


In [ ]:
df_new2021 = df_new.loc[(df_new['Job Created'] <= "2022-01-01 00:00:00.000000") & (df_new['Job Created'] >= "2021-01-01 00:00:00.000000")]

df_new2021_1 = df_new.loc[(df_new['Job Created'] <= "2021-02-01 00:00:00.000000") & (df_new['Job Created'] >= "2021-01-01 00:00:00.000000")]
df_new2021_2 = df_new.loc[(df_new['Job Created'] <= "2021-03-01 00:00:00.000000") & (df_new['Job Created'] >= "2021-02-01 00:00:00.000000")]
df_new2021_3 = df_new.loc[(df_new['Job Created'] <= "2021-04-01 00:00:00.000000") & (df_new['Job Created'] >= "2021-03-01 00:00:00.000000")]
df_new2021_4 = df_new.loc[(df_new['Job Created'] <= "2021-05-01 00:00:00.000000") & (df_new['Job Created'] >= "2021-04-01 00:00:00.000000")]
df_new2021_5 = df_new.loc[(df_new['Job Created'] <= "2021-06-01 00:00:00.000000") & (df_new['Job Created'] >= "2021-05-01 00:00:00.000000")]
df_new2021_6 = df_new.loc[(df_new['Job Created'] <= "2021-07-01 00:00:00.000000") & (df_new['Job Created'] >= "2021-06-01 00:00:00.000000")]
df_new2021_7 = df_new.loc[(df_new['Job Created'] <= "2021-08-01 00:00:00.000000") & (df_new['Job Created'] >= "2021-07-01 00:00:00.000000")]
df_new2021_8 = df_new.loc[(df_new['Job Created'] <= "2021-09-01 00:00:00.000000") & (df_new['Job Created'] >= "2021-08-01 00:00:00.000000")]
df_new2021_9 = df_new.loc[(df_new['Job Created'] <= "2021-10-01 00:00:00.000000") & (df_new['Job Created'] >= "2021-09-01 00:00:00.000000")]
df_new2021_10 = df_new.loc[(df_new['Job Created'] <= "2021-11-01 00:00:00.000000") & (df_new['Job Created'] >= "2021-10-01 00:00:00.000000")]
df_new2021_11 = df_new.loc[(df_new['Job Created'] <= "2021-12-01 00:00:00.000000") & (df_new['Job Created'] >= "2021-11-01 00:00:00.000000")]
df_new2021_12 = df_new.loc[(df_new['Job Created'] <= "2022-01-01 00:00:00.000000") & (df_new['Job Created'] >= "2021-12-01 00:00:00.000000")]

df2021j = len(df_new2021_1)
df2021f = len(df_new2021_2)
df2021m = len(df_new2021_3)
df2021a = len(df_new2021_4)
df2021ma = len(df_new2021_5)
df2021ju = len(df_new2021_6)
df2021jy = len(df_new2021_7)
df2021au = len(df_new2021_8)
df2021s = len(df_new2021_9)
df2021o = len(df_new2021_10)
df2021n = len(df_new2021_11)
df2021d = len(df_new2021_12)
print(df2021j, df2021f, df2021m, df2021a, df2021ma, df2021ju, df2021jy, df2021au, df2021s, df2021o, df2021n, df2021d)

1801 2211 1394 1169 1021 962 879 1163 1013 1111 752 359


In [ ]:
df_new2022 = df_new.loc[(df_new['Job Created'] <= "2023-01-01 00:00:00.000000") & (df_new['Job Created'] >= "2022-01-01 00:00:00.000000")]

df_new2022_1 = df_new.loc[(df_new['Job Created'] <= "2022-02-01 00:00:00.000000") & (df_new['Job Created'] >= "2022-01-01 00:00:00.000000")]
df_new2022_2 = df_new.loc[(df_new['Job Created'] <= "2022-03-01 00:00:00.000000") & (df_new['Job Created'] >= "2022-02-01 00:00:00.000000")]
df_new2022_3 = df_new.loc[(df_new['Job Created'] <= "2022-04-01 00:00:00.000000") & (df_new['Job Created'] >= "2022-03-01 00:00:00.000000")]
df_new2022_4 = df_new.loc[(df_new['Job Created'] <= "2022-05-01 00:00:00.000000") & (df_new['Job Created'] >= "2022-04-01 00:00:00.000000")]
df_new2022_5 = df_new.loc[(df_new['Job Created'] <= "2022-06-01 00:00:00.000000") & (df_new['Job Created'] >= "2022-05-01 00:00:00.000000")]
df_new2022_6 = df_new.loc[(df_new['Job Created'] <= "2022-07-01 00:00:00.000000") & (df_new['Job Created'] >= "2022-06-01 00:00:00.000000")]
df_new2022_7 = df_new.loc[(df_new['Job Created'] <= "2022-08-01 00:00:00.000000") & (df_new['Job Created'] >= "2022-07-01 00:00:00.000000")]
df_new2022_8 = df_new.loc[(df_new['Job Created'] <= "2022-09-01 00:00:00.000000") & (df_new['Job Created'] >= "2022-08-01 00:00:00.000000")]
df_new2022_9 = df_new.loc[(df_new['Job Created'] <= "2022-10-01 00:00:00.000000") & (df_new['Job Created'] >= "2022-09-01 00:00:00.000000")]
df_new2022_10 = df_new.loc[(df_new['Job Created'] <= "2022-11-01 00:00:00.000000") & (df_new['Job Created'] >= "2022-10-01 00:00:00.000000")]
df_new2022_11 = df_new.loc[(df_new['Job Created'] <= "2022-12-01 00:00:00.000000") & (df_new['Job Created'] >= "2022-11-01 00:00:00.000000")]
df_new2022_12 = df_new.loc[(df_new['Job Created'] <= "2023-01-01 00:00:00.000000") & (df_new['Job Created'] >= "2022-12-01 00:00:00.000000")]

df2022j = len(df_new2022_1)
df2022f = len(df_new2022_2)
df2022m = len(df_new2022_3)
df2022a = len(df_new2022_4)
df2022ma = len(df_new2022_5)
df2022ju = len(df_new2022_6)
df2022jy = len(df_new2022_7)
df2022au = len(df_new2022_8)
df2022s = len(df_new2022_9)
df2022o = len(df_new2022_10)
df2022n = len(df_new2022_11)
df2022d = len(df_new2022_12)
print(df2022j, df2022f, df2022m, df2022a, df2022ma, df2022ju, df2022jy, df2022au, df2022s, df2022o, df2022n, df2022d)

839 1196 1223 1063 1159 1336 1094 999 1078 1269 1067 621


In [22]:
def df_to_X_y(df1, window_size=5):

  X = []
  y = []

  for i in range(len(df1)-window_size):
    row = [[a] for a in df1[i:i+window_size]]
    X.append(row)
    label = df1[i+window_size]
    y.append(label)

  return np.array(X), np.array(y)

In [24]:
Servers = df1['Servers']
WINDOW_SIZE = 5
X1, y1 = df_to_X_y(Servers, WINDOW_SIZE)
X1.shape, y1.shape

((48, 5, 1), (48,))

In [30]:
num_total_samples = len(X1)
num_train_samples = int(0.6 * num_total_samples)
num_val_samples = int(0.2 * num_total_samples)

X_train1, y_train1 = X1[:num_train_samples], y1[:num_train_samples]
X_val1, y_val1 = X1[num_train_samples:num_train_samples + num_val_samples], y1[num_train_samples:num_train_samples + num_val_samples]
X_test1, y_test1 = X1[num_train_samples + num_val_samples:], y1[num_train_samples + num_val_samples:]

print(X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape)

(28, 5, 1) (28,) (9, 5, 1) (9,) (11, 5, 1) (11,)


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

model1 = Sequential()
model1.add(InputLayer((5, 1)))
model1.add(LSTM(64))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))
model1.summary()

#model1 = Sequential()
#model1.add(LSTM(64, input_shape=(5, 1)))
#model1.add(Dense(1, activation='linear'))

#model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                16896     
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 17425 (68.07 KB)
Trainable params: 17425 (68.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
cp1 = ModelCheckpoint('model1/', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])

In [33]:
model1.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=10, callbacks=[cp1])

Epoch 1/10
1/1 [==============================] - 8s 8s/step - loss: 883731.3125 - root_mean_squared_error: 940.0698 - val_loss: 302891.8438 - val_root_mean_squared_error: 550.3561
Epoch 2/10
1/1 [==============================] - 4s 4s/step - loss: 883714.3125 - root_mean_squared_error: 940.0608 - val_loss: 302884.7500 - val_root_mean_squared_error: 550.3497
Epoch 3/10
1/1 [==============================] - 4s 4s/step - loss: 883697.5000 - root_mean_squared_error: 940.0519 - val_loss: 302877.6562 - val_root_mean_squared_error: 550.3432
Epoch 4/10
1/1 [==============================] - 5s 5s/step - loss: 883680.9375 - root_mean_squared_error: 940.0430 - val_loss: 302870.6250 - val_root_mean_squared_error: 550.3369
Epoch 5/10
1/1 [==============================] - 4s 4s/step - loss: 883664.5000 - root_mean_squared_error: 940.0343 - val_loss: 302863.1875 - val_root_mean_squared_error: 550.3301
Epoch 6/10
1/1 [==============================] - 4s 4s/step - loss: 883648.5000 - root_mean_sq

In [34]:
from tensorflow.keras.models import load_model
model1 = load_model('model1/')

In [35]:
train_predictions = model1.predict(X_train1).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train1})
train_results

1/1 [==============================] - 1s 507ms/step


,Train Predictions,Actuals
0,0.253419,1541
1,0.272904,1085
2,0.274659,1505
3,0.266743,1193
4,0.275335,1407
5,0.278340,878
6,0.279045,835
7,0.273740,593
8,0.257770,1184
9,0.243172,1265
